# Practice with the Python ATProtoSDK
This ipython notebook will walk you through the basics of working with the
ATProto python sdk. The content here heavily draws on [these examples](https://github.com/MarshalX/atproto/tree/main/examples)

In [38]:
from atproto import Client
from dotenv import load_dotenv
import os
import pprint


load_dotenv(override=True)
USERNAME = os.getenv("USERNAME")
PW = os.getenv("PW")

## Logging into your account

In [2]:
client = Client()
profile = client.login(USERNAME, PW)
pprint.pprint(profile.__dict__)

{'associated': ProfileAssociated(chat=None, feedgens=0, labeler=True, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'),
 'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:yzpplgm5kftdgpf2wsnrbgdn/bafkreih3fpryxoepb44fzyr3sfn32fr7fqqka4kle6h4not7jlwtdvzghe@jpeg',
 'banner': None,
 'created_at': '2025-02-13T17:12:12.845Z',
 'description': None,
 'did': 'did:plc:yzpplgm5kftdgpf2wsnrbgdn',
 'display_name': '',
 'followers_count': 1,
 'follows_count': 1,
 'handle': 'trustylabeler.bsky.social',
 'indexed_at': '2025-02-13T17:12:12.845Z',
 'joined_via_starter_pack': None,
 'labels': [],
 'pinned_post': None,
 'posts_count': 0,
 'py_type': 'app.bsky.actor.defs#profileViewDetailed',
 'viewer': ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState')}


## Working with posts

In [3]:
def get_latest_posts(client, limit=100):
    feed = client.app.bsky.feed.get_timeline({'limit': limit})
    posts = feed['feed']
    return [{
        'text': post['post']['record']['text'],
        'uri': post['post']['uri'],
        'cid': post['post']['cid'],
        'handle': post['post']['author']['handle']
    } for post in posts]

In [4]:
response = client.app.bsky.feed.search_posts({'q': 'giveaway', 'limit': 10})

In [5]:
response

Response(posts=[PostView(author=ProfileViewBasic(did='did:plc:zyeaqucmyejzsyb5ziad3pvo', handle='civicchampion.bsky.social', associated=None, avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:zyeaqucmyejzsyb5ziad3pvo/bafkreifslzvci47rqqzms6cwytck4i6f33cxktiwt5bvtfsvzcov4pbypa@jpeg', created_at='2025-02-03T10:15:14.044Z', display_name='Schaden Freude (FAFO Central)', labels=[], viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewBasic'), cid='bafyreif4x5ud5jj74wyjbhcqnysq4lvkjvaxmd2zjx6ls6khx2klsxlhqu', indexed_at='2025-04-17T20:17:59.585Z', record=Record(created_at='2025-04-17T20:17:55.581Z', text='Schadenfreude: The holier-than-thou demeanor is a dead giveaway.', embed=Main(images=[Image(alt='', image=BlobRef(mime_type='image/jpeg', size=342811, ref=IpldLink(link='bafkreicxck4s3xlluo7udre

In [6]:
get_latest_posts(client, 10)

[{'text': '📢\xa0App Version 1.100 is rolling out now (2/2)\n\nTrying to find more on Bluesky? The search page is now "Explore," with updated trends, suggested accounts, and more!',
  'uri': 'at://did:plc:z72i7hdynmk6r22z27h6tvur/app.bsky.feed.post/3lmi3hmjsak24',
  'cid': 'bafyreiax6jqg2mg32yeyxb6bz42pwbd6mtqoxeo6rp6e7wwfhw2ef47dh4',
  'handle': 'bsky.app'},
 {'text': '📢\xa0App Version 1.100 is rolling out now (1/2)\n\nChat reactions are here! You can now respond to chat messages (aka “direct messages”) with an emoji ❤️',
  'uri': 'at://did:plc:z72i7hdynmk6r22z27h6tvur/app.bsky.feed.post/3lmi3hlglc224',
  'cid': 'bafyreihbk62hhazqtq5oyi66nn2hpayjq6n5pwjhskceoxvwoww6hlo6ba',
  'handle': 'bsky.app'},
 {'text': 'Wish that you spent your Friday doing something besides checking if TikTok was banned or not? \n\nTry out @skylight.social — a video app built upon the same open network as Bluesky. You can login with your Bluesky account, and all of your followers seamlessly come with you.',
  'u

In [4]:
def post_from_url(client: Client, url: str):
    """
    Retrieve a Bluesky post from its URL
    """
    parts = url.split("/")
    rkey = parts[-1]
    handle = parts[-3]
    return client.get_post(rkey, handle)

post = post_from_url(client, "https://bsky.app/profile/labeler-test.bsky.social/post/3lktj7ewxxv2q")
pprint.pprint(post.value.__dict__)

{'created_at': '2025-03-20T20:14:57.103160+00:00',
 'embed': Main(images=[Image(alt='dog', image=BlobRef(mime_type='image/jpeg', size=169278, ref=IpldLink(link='bafkreibahplioamouecglrcqnshcxzdrwawdtwl5h676d2l7k7xbbti3pa'), py_type='blob'), aspect_ratio=None, py_type='app.bsky.embed.images#image')], py_type='app.bsky.embed.images'),
 'entities': None,
 'facets': None,
 'labels': None,
 'langs': ['en'],
 'py_type': 'app.bsky.feed.post',
 'reply': None,
 'tags': None,
 'text': 'check out this dog!'}


In [5]:
post.value.text

'check out this dog!'

In [6]:
# https://github.com/MarshalX/atproto/blob/main/examples/profile_posts.py
prof_feed = client.get_author_feed(actor="weratedogs.com")
for i, feed_view in enumerate(prof_feed.feed[:10]):
    print(f"Post {i}:", feed_view.post.record.text)

post = prof_feed.feed[0].post
likes_resp = client.get_likes(post.uri, post.cid, limit=10)
print("Likes:", [like.actor.handle for like in likes_resp.likes])

post_thread_resp = client.get_post_thread(post.uri)
print([rep.post.record.text for rep in post_thread_resp.thread.replies[:10]])

Post 0: This is Sigourney Weaver and Alice. Sigourney Weaver is hard at work digging a hole, and Alice has accepted her very avoidable fate. 13/10 for both
Post 1: With your help, we sponsored the care Fiyero needed for his seizures. His foster family also officially adopted him!
 Fiyero’s been a champ at every check-up and is adjusting nicely to his medication. Thanks to you, he’s set for life with humans who already adore him ❤️‍🩹
Post 2: CONGRATS FIYERO ❤️
Post 3: Marley became one with the burger. 🍔  😂
Post 4: This is Marley. He does not simply eat his hamburgers. He inhales them. 13/10
Post 5: 🚨 FLIPPER GOT ADOPTED!!! 🚨

She also can’t stop showing off her front legs, which you helped straighten and heal. These days, she puts them to good use wrestling with her new sister, Sweetie. According to her rescue, it was love at first sight. Thanks for giving this girl her happily ever after ❤️‍🩹
Post 6: new best friends right there!
Post 7: You need this little hop
Post 8: This is Wallac

## Followers/following

How might you use this information to investigate/mitigate a harm?

In [7]:

follower_resp = client.get_followers("weratedogs.com", limit=10)
following_resp = client.get_follows("weratedogs.com", limit=10)
print("Followers:", [follower.handle for follower in follower_resp.followers])
print("Following:", [follow.handle for follow in following_resp.follows])



Followers: ['kelly3010.bsky.social', 'homaksu.bsky.social', 'jennifersamule.bsky.social', 'walkerbn.bsky.social', 'cuneyterdem0.bsky.social', 'shinddha.bsky.social', 'yooperann.bsky.social', 'tor37.bsky.social', 'windowtothesoull.bsky.social', 'twocakesup.bsky.social']
Following: ['15outof10.org']


## Exercise: Compute average dog ratings
The WeRateDogs account includes ratings out of 10 within some of its posts.
Write a script that computes the average rating (out of 10) for the 100 most
recent posts from this account. (note that not every post will have a rating)

In [9]:
import re

In [6]:
import pandas as pd 
import re
import time 
import json 
from atproto import Client
from dotenv import load_dotenv
import os 

load_dotenv(override=True)
USERNAME = os.getenv("USERNAME")
PW = os.getenv("PW")

client = Client()
client.login(USERNAME, PW)

df_keywords = pd.read_csv("./giveaway-labeler/giveaway-words.csv")
giveaway_words = df_keywords['word'].dropna().tolist()
cta_words = df_keywords['call-to-action'].dropna().tolist()

df_urls = pd.read_csv("./bluesky_giveaway_labels.csv")
post_urls = df_urls['URL'].dropna().tolist()

confirmed_matches = []

for url in post_urls:
    try:
        parts = url.split('/post/')
        did = parts[0].split('/')[-1]
        rkey = parts[1]

        post = client.com.atproto.repo.get_record({
            "repo": did,
            "collection": "app.bsky.feed.post",
            "rkey": rkey
        })

        text = post['value'].get('text', '')

        has_giveaway = any(re.search(rf"\b{re.escape(word)}\b", text, re.IGNORECASE) for word in giveaway_words)
        has_cta = any(re.search(rf"\b{re.escape(cta)}\b", text, re.IGNORECASE) for cta in cta_words)

        if has_giveaway and has_cta:
            actor_info = client.app.bsky.actor.get_profile({'actor':did})
            confirmed_matches.append({
                "url": url,
                "did": did,
                "rkey": rkey,
                "text": text,
                "followers_count": actor_info.get("followersCount"),
                "follows_count": actor_info.get("followsCount"),
                "posts_count": actor_info.get("postsCount"),
                "created_at": actor_info.get("createdAt"),
            })

            time.sleep(0.4)

    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        continue

print(f"Confirmed {len(confirmed_matches)} posts with BOTH a giveaway word and a CTA.")

ModuleNotFoundError: No module named 'atproto'

In [10]:
def extract_score(text):
    # This pattern looks for:
    # \d+ - one or more digits
    # \s* - optional whitespace
    # / - literal forward slash
    # \s* - optional whitespace
    # 10 - literal "10"
    pattern = r'(\d+)\s*/\s*10'
    match = re.search(pattern, text)
    if match:
        return int(match.group(1))
    return None

In [13]:
def compute_avg_dog_rating(num_posts):
    # TODO: complete
    total_score = 0
    num_scores = 0
    for i, feed_view in enumerate(prof_feed.feed[:num_posts]):
        score = extract_score(feed_view.post.record.text)
        if score is not None:
            # print(f"Found score: {score}/10")
            total_score += score
            num_scores += 1
        # else:
            # print("No score found in the format X/10")
    return total_score / num_scores if num_scores > 0 else 0

print("The average rating is:", compute_avg_dog_rating(100))

The average rating is: 13.0


## Exercise: Dog names
Collect the names of dogs within the latest 100 posts and print them to the
console. Hint: see if you can identify a pattern in the posts.

In [15]:
def extract_names(text):
    # This pattern looks for:
    # (?<![\.\?\!]\s) - negative lookbehind for period/question mark/exclamation followed by whitespace
    # (?<!\A) - negative lookbehind for start of string
    # \b[A-Z][a-zA-Z]*\b - word boundary, capital letter, followed by any letters, word boundary
    pattern = r'(?<![\.\?\!]\s)(?<!\A)\b[A-Z][a-zA-Z]*\b'
    
    matches = re.finditer(pattern, text)
    return [match.group() for match in matches]

In [16]:
def collect_dog_names(num_posts):
    # TODO: complete
    all_names = []
    for i, feed_view in enumerate(prof_feed.feed[:num_posts]):
        names = extract_names(feed_view.post.record.text)
        if names:
            all_names.extend(names)
    return all_names

print("Here are the dog_names:", collect_dog_names(100))

Here are the dog_names: ['George', 'SeniorPupSaturday', 'Fiyero', 'Dasia', 'The', 'Farmers', 'Dog', 'Top', 'Dogs', 'Dill', 'GIANT', 'Roger', 'Roger', 'ER', 'Panda', 'We', 'Panda', 'Panda', 'I', 'Ellie', 'Flipper', 'ALL', 'Good', 'Lucie', 'Muamba', 'Ollie', 'Heckles', 'ONLY', 'April', 'Top', 'WORST', 'Dogs', 'Joey', 'Bambi', 'Top', 'Dogs', 'March', 'Pippa', 'SeniorPupSaturday', 'Thank', 'Butterfly', 'Butterfly', 'Top', 'Dogs', 'Leon', 'Hubie', 'Fig', 'Oreo', 'FBI', 'Choco', 'We', 'ADOPTED', 'Choco', 'Rex']


## Exercise: Soliciting donations
Some posts from the WeRateDogs account ask for donations -- usually for
covering medical costs for the featured dogs. Within the latest 100 posts, print
the text content of those that fall into this category

In [ ]:
def donation_posts(num_posts):
    # TODO: complete
    return []

for post_text in donation_posts(100):
    print(post_text)